**Introductory and intermediate computing for Data Science [Barcelona School of Economics]**

`Instructor:` Maxim Fedotov  
`Program:` M.Sc. in Data Science Methodology

# Class 4

In [19]:
from math import pi, log
from typing import Union

def log_normal_pdf(x: Union[float, int], mu: Union[float, int] = 0, sigma_sq: Union[float, int] = 1) -> float:
    """
        Computes pdf of a normal distribution with parameters mu and sigma_sq at point x.
        
        args:
            x:         concrete point
            mu:        mean parameter of the distribution
            sigma_sq:  variance parameter of the distribution
        
        returns:
            A value of pdf.
    """
    if sigma_sq <= 0:
        raise ValueError("The parameter sigma_sq has to be posititve.")
    return -(x - mu)**2 / (2 * sigma_sq) - 0.5 * log(2 * pi * sigma_sq)

log_normal_pdf(0)

-0.9189385332046727

In [7]:
x_vals = [1, 2, 3, -1, -2, -3]
mu_vals = [2] * 3 + [-2] * 3
sigma_sq_vals = [0.5, 0.5, 0.3, 1, 2, 0.3]

for x, mu, sigma_sq in zip(x_vals, mu_vals, sigma_sq_vals):
    log_pdf = log_normal_pdf(x, mu,sigma_sq)
    print(f"The pdf of a normal with mu = {mu}, sigma = {sigma_sq} at x = {x} is {log_pdf}")

The pdf of a normal with mu = 2, sigma = 0.5 at x = 1 is -1.5723649429247
The pdf of a normal with mu = 2, sigma = 0.5 at x = 2 is -0.5723649429247001
The pdf of a normal with mu = 2, sigma = 0.3 at x = 3 is -1.9836187977083715
The pdf of a normal with mu = -2, sigma = 1 at x = -1 is -1.4189385332046727
The pdf of a normal with mu = -2, sigma = 2 at x = -2 is -1.2655121234846454
The pdf of a normal with mu = -2, sigma = 0.3 at x = -3 is -1.9836187977083715


In [8]:
from itertools import product

product(x_vals, mu_vals, sigma_sq_vals)

## Map

In [20]:
list(map(log_normal_pdf, x_vals))

[-1.4189385332046727,
 -2.9189385332046727,
 -5.418938533204672,
 -1.4189385332046727,
 -2.9189385332046727,
 -5.418938533204672]

## Lambda

In [22]:
female_indicator = lambda x: 1 if x == 'female' else 0

list(map(female_indicator, ["female", "male", "female", "male"]))

[1, 0, 1, 0]

In [21]:
# or other technical function

param_combinations = zip(x_vals, mu_vals, sigma_sq_vals)
list(map(lambda _tuple: log_normal_pdf(*_tuple), param_combinations))

[-1.5723649429247,
 -0.5723649429247001,
 -1.9836187977083715,
 -1.4189385332046727,
 -1.2655121234846454,
 -1.9836187977083715]